# Foundations of Artificial Intelligence and Machine Learning
## A Program by IIIT-H and TalentSprint
#### To be done in the Lab

The objective of this experiment is to understand regularization.

<img src="reg1.png">

While our loss function allows us to determine how well (or poorly) our set of parameters (i.e., weight matrix, and bias vector) are performing on a given classification task, the loss function itself does not take into account how the weight matrix “looks”. This brings us to the following questions.  How do we go about choosing a set of parameters that will help ensure our model generalizes well? Or at the very least, lessen the affects of overfitting?

The answer is <b>regularization.</b>

This notebook is divided into two parts. In the first part we will observe the behaviour of the weight space after adding a regularization term while in the second part, we will use some property of the learned weight space to remove redundant connections.



##### Keywords

* L1 Regularization
* Pruning
* Batchnorm

##### Expected time to complete the experiment is : 90min

Let us begin by our regular import statements.

In [1]:
!pip install torch torchvision

In [ ]:
!pip3 install http://download.pytorch.org/whl/cpu/torch-0.4.1-cp36-cp36m-win_amd64.whl 


In [4]:
import numpy as np
import torch 
import torchvision
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
%matplotlib inline
import matplotlib.pyplot as plt

ImportError: DLL load failed: The specified module could not be found.

### Hyperparameters to be used

In [5]:
num_epochs = 5
batch_size = 100
learning_rate = 0.001
use_reg = False
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Downloading the MNIST Dataset

In [ ]:
train_dataset = dsets.MNIST(root='../data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='../data',
                           train=False, 
                           transform=transforms.ToTensor())

### Dataloader

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

### Defining the network

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc1 = nn.Linear(7*7*32, 300)
        self.fc2 = nn.Linear(300, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

<b>The below function is called to reinitialize the weights of the network and define the required loss criterion and the optimizer.</b> 

In [ ]:
def reset_model():
    net = Net()
    net = net.to(device)

    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    return net,criterion,optimizer

### L1 Regularization

Here, we define a L1 Regularizer and add it to our loss function. The L1 Regularization term basically adds a penalty, equivalent to the absolute value of the magnitude of the weights. This ensures that the magnitude of the weights do not become too high.

We have seen in the previous lectures that adding the L1 regularizer ensures sparsity. This is important becuase our ultimate aim is to prune connections from our network. That would mean we should try to make as many weights extremely close to 0 as possible. Thus, adding this penalty term ensures sparsity. The L1 Regularizer is also known as Lass Regression and it can be defined as follows:

$$P=\lambda*\sum_{i=1}^{n}|\theta_{i}-0|$$

In [ ]:
def l1_regularizer(net, loss, beta):
    l1_crit = nn.L1Loss(size_average=False)
    reg_loss = 0
    for param in net.parameters():
        target = (torch.FloatTensor(param.size()).zero_()).to(device)
        reg_loss += l1_crit(param, target)
        
    loss += beta * reg_loss
    return loss

### Initializing the model

In [ ]:
net, criterion, optimizer = reset_model()

### Defining the training function

In [ ]:
# Train the Model

def training(net, reset = True):
    if reset == True:
        net, criterion, optimizer = reset_model()
    else:
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    
    net.train()
    for epoch in range(num_epochs):
        total_loss = 0
        accuracy = []
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            temp_labels = labels
            

            # Forward + Backward + Optimize
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)

            if use_reg == True :
                loss = l1_regularizer(net,loss,beta=0.001)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == temp_labels).sum().item()
            accuracy.append(correct/float(batch_size))

        print('Epoch: %d, Loss: %.4f, Accuracy: %.4f' %(epoch+1,total_loss, (sum(accuracy)/float(len(accuracy)))))
    
    return net

### Defining the testing function

In [ ]:
# Test the Model
def testing(net):
    net.eval() 
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()




    print('Test Accuracy of the network on the 10000 test images: %.2f %%' % (100.0 * correct / total))

### Training and testing the network

In [ ]:
reset = True
net = training(net, reset)
testing(net)

### Function for plotting the weight distribution

We want to plot the weights learnt during training and we define the following function to do so. It is important to note that in order ro achieve smoothness in the behaviour of the plot we use 256 clusters here where the nearest weight values are rounded to. Feel free to experiment with defining lesser number of clusters for the weight distribution approximation. However, the plot may not be as smooth. 

We plot the cluster values in the x-axis and the frequency of the weights in the y-axis.

In [ ]:
def weightdistribution(weights):
    maxim= np.amax(weights)
    print("Maximum value of learnt weights: " + str(maxim))
    
    minim= np.amin(weights)
    print("Minimum value of learnt weights: " + str(minim))
    
    step= (maxim-minim)/255
    freq= np.zeros(256)
    steps=[]

    for i in range(0,256):
        steps.append(minim)
        minim+=step
    
    m = weights.shape[0]
    
    for i in range(0,m):
        e= weights[i]
        dist= (steps-e)**2
        freq[np.argmin(dist)]+=1
    
        
    plt.plot(steps,list(freq))
    plt.show()

In [ ]:
weightdistribution(net.state_dict()['layer2.0.weight'].cpu().numpy().flatten())
weightdistribution(net.state_dict()['fc1.weight'].cpu().numpy().flatten())
weightdistribution(net.state_dict()['fc2.weight'].cpu().numpy().flatten())

### Defining the pruning function

We use a threshold based criteria to remove certain connections in the networks which fall within a threshold value. Hence, we define a function which takes the network as input and a vlaue for threshold and forces those connections to be 0 which fall with the threshold given by the following equation.

$$
\begin{equation}
 W_{i} = 
 \begin{cases} 
      W_{i} & W_{i} < -\theta \\
      0 & -\theta\leq W_{i}\leq \theta \\
      W_{i} & \theta < W_{i} 
  \end{cases}
\end{equation}
$$

We prune both the weights and biases of Convolution layers, Fully Connected layers and the Batch Normalization layers

In [ ]:
def prune_weight(net, threshold):
    
    for m in net.modules():
        if isinstance(m,nn.Conv2d) or isinstance(m,nn.BatchNorm2d) or isinstance(m,nn.Linear):

            temp_weight = m.weight.data.cpu().numpy()
            dims = temp_weight.shape
            print('WEIGHT ',dims)
            print("Before pruning------->")
            print(np.count_nonzero(temp_weight))

            temp_weight = temp_weight.flatten()

            [x1,x2]=((np.where(np.all([[(-1*threshold) < (temp_weight)] , [(temp_weight) < threshold]],axis=0))))
            temp_weight[x2] = 0
            temp_weight = np.reshape(temp_weight,dims)
            print("After pruning------->")
            print(np.count_nonzero(temp_weight))
            print('-------------------------------------------------------------------------------')
            m.weight.data = (torch.FloatTensor(temp_weight).cuda())

            temp_bias = m.bias.data.cpu().numpy()
            dims = temp_bias.shape
            print('BIAS ',dims)
            print("Before pruning------->")
            print(np.count_nonzero(temp_bias))

            temp_bias = temp_bias.flatten()

            [x1,x2]=((np.where(np.all([[(-1*threshold) < (temp_bias)] , [(temp_bias) < threshold]],axis=0))))
            temp_bias[x2] = 0
            temp_bias = np.reshape(temp_bias,dims)
            print("After pruning------->")
            print(np.count_nonzero(temp_bias))
            print('-------------------------------------------------------------------------------')
            m.bias.data = (torch.FloatTensor(temp_bias).cuda())

# for m in net.modules():
#     if isinstance(m,nn.Conv2d):
#         print m.bias.data
  

Here, we check how many connetions have been pruned

In [ ]:
threshold = 0.01
prune_weight(net,threshold)

Training the network again using the regularizer. The loss is modified as 

$$
loss = loss + regularized_loss
$$

In [ ]:
reset = True
use_reg = True
net = training(net, reset)
testing(net)

<b>Visualizing the weight distributions with the regularized loss</b>

In [ ]:
weightdistribution(net.state_dict()['layer2.0.weight'].cpu().numpy().flatten())
weightdistribution(net.state_dict()['fc1.weight'].cpu().numpy().flatten())
weightdistribution(net.state_dict()['fc2.weight'].cpu().numpy().flatten())

By using the L1 Regularizer, we see that we have forced the network to learn weights within a constrained subspace. Since more number of weights are closer to 0, it is evident that we will be able to make more weights 0 by using the same threshold as before. This, therefore ensures sparsity in every layer of the network.

An interesting observation is that the netowrk tells us that it does not require biases for certain layers.

In [ ]:
threshold = 0.01
prune_weight(net,threshold)

### References

1. http://www.pyimagesearch.com/2016/09/19/understanding-regularization-for-image-classification-and-machine-learning/